In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df = pd.read_csv('/content/nba_games_all.csv')
df.head()

,game_id,game_date,matchup,team_id,is_home,wl,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,a_team_id,season_year,season_type,season
0,20800741,2009-02-06,SAC vs. UTA,1610612762,f,W,29.0,22.0,0.569,240,39.0,74.0,0.527,4.0,13.0,0.308,29.0,37.0,1.0,8.0,36.0,44,19.0,5.0,4.0,18.0,26.0,111,1610612758,2008,Regular Season,2008-09
1,20800701,2009-01-31,POR vs. UTA,1610612762,f,L,26.0,22.0,0.542,240,37.0,68.0,0.544,5.0,20.0,0.250,29.0,37.0,1.0,7.0,20.0,27,17.0,6.0,0.0,15.0,22.0,108,1610612757,2008,Regular Season,2008-09
2,20800584,2009-01-16,MEM vs. UTA,1610612762,f,W,24.0,16.0,0.600,240,37.0,71.0,0.521,7.0,19.0,0.368,20.0,27.0,1.0,7.0,26.0,33,23.0,9.0,3.0,15.0,22.0,101,1610612763,2008,Regular Season,2008-09
3,20800558,2009-01-12,IND @ UTA,1610612762,t,W,23.0,15.0,0.605,240,39.0,85.0,0.459,8.0,18.0,0.444,34.0,37.0,1.0,9.0,31.0,40,24.0,10.0,6.0,8.0,20.0,120,1610612754,2008,Regular Season,2008-09
4,20800440,2008-12-27,HOU vs. UTA,1610612762,f,L,18.0,14.0,0.563,290,49.0,100.0,0.490,5.0,13.0,0.385,12.0,18.0,1.0,11.0,29.0,40,35.0,13.0,7.0,9.0,27.0,115,1610612745,2008,Regular Season,2008-09


In [5]:
copied = df.copy(deep=True)
copied.head()

,game_id,game_date,matchup,team_id,is_home,wl,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,a_team_id,season_year,season_type,season
0,20800741,2009-02-06,SAC vs. UTA,1610612762,f,W,29.0,22.0,0.569,240,39.0,74.0,0.527,4.0,13.0,0.308,29.0,37.0,1.0,8.0,36.0,44,19.0,5.0,4.0,18.0,26.0,111,1610612758,2008,Regular Season,2008-09
1,20800701,2009-01-31,POR vs. UTA,1610612762,f,L,26.0,22.0,0.542,240,37.0,68.0,0.544,5.0,20.0,0.250,29.0,37.0,1.0,7.0,20.0,27,17.0,6.0,0.0,15.0,22.0,108,1610612757,2008,Regular Season,2008-09
2,20800584,2009-01-16,MEM vs. UTA,1610612762,f,W,24.0,16.0,0.600,240,37.0,71.0,0.521,7.0,19.0,0.368,20.0,27.0,1.0,7.0,26.0,33,23.0,9.0,3.0,15.0,22.0,101,1610612763,2008,Regular Season,2008-09
3,20800558,2009-01-12,IND @ UTA,1610612762,t,W,23.0,15.0,0.605,240,39.0,85.0,0.459,8.0,18.0,0.444,34.0,37.0,1.0,9.0,31.0,40,24.0,10.0,6.0,8.0,20.0,120,1610612754,2008,Regular Season,2008-09
4,20800440,2008-12-27,HOU vs. UTA,1610612762,f,L,18.0,14.0,0.563,290,49.0,100.0,0.490,5.0,13.0,0.385,12.0,18.0,1.0,11.0,29.0,40,35.0,13.0,7.0,9.0,27.0,115,1610612745,2008,Regular Season,2008-09


In [4]:
def preprocess(df):
  df = df.dropna()
  # Normalize Data: x - xmin / xmax - xmin
  arr = ['team_id', 'w_pct', 'min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a',
         'fg3_pct', 'ftm', 'fta', 'ft_pct', 'a_team_id']
  for field in arr:
    scaler = MinMaxScaler() 
    df[field] = scaler.fit_transform(df[[field]])
  df = df.replace('W', 1)
  df = df.replace('L', 0)
  df = df.replace('t', 1)
  df = df.replace('f', 0)
  df['wl'] = df['wl'].astype('int') 
  df['is_home'] = df['is_home'].astype('int') 
  return df

In [8]:
def ind(df):
  d = {}
  i = 0
  for index, row in df.iterrows():
    key = row['MATCHUP']
    val = 0
    if key in d:
      val = d[key]
    else:
      d[key] = i
      val = i
      i += 1
    df.loc[index, 'MATCHUP'] = val
  return df

In [6]:
# df = ind(df)
df = preprocess(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [7]:
df.head()

,game_id,game_date,matchup,team_id,is_home,wl,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,a_team_id,season_year,season_type,season
0,20800741,2009-02-06,SAC vs. UTA,0.006102,0,1,29.0,22.0,0.569,0.193548,0.338235,0.234694,0.643182,0.129032,0.151899,0.308,0.549020,0.571429,1.0,8.0,36.0,44,19.0,5.0,4.0,18.0,26.0,111,0.005126,2008,Regular Season,2008-09
1,20800701,2009-01-31,POR vs. UTA,0.006102,0,0,26.0,22.0,0.542,0.193548,0.308824,0.173469,0.681818,0.161290,0.240506,0.250,0.549020,0.571429,1.0,7.0,20.0,27,17.0,6.0,0.0,15.0,22.0,108,0.004882,2008,Regular Season,2008-09
2,20800584,2009-01-16,MEM vs. UTA,0.006102,0,1,24.0,16.0,0.600,0.193548,0.308824,0.204082,0.629545,0.225806,0.227848,0.368,0.372549,0.412698,1.0,7.0,26.0,33,23.0,9.0,3.0,15.0,22.0,101,0.006346,2008,Regular Season,2008-09
3,20800558,2009-01-12,IND @ UTA,0.006102,1,1,23.0,15.0,0.605,0.193548,0.338235,0.346939,0.488636,0.258065,0.215190,0.444,0.647059,0.571429,1.0,9.0,31.0,40,24.0,10.0,6.0,8.0,20.0,120,0.004149,2008,Regular Season,2008-09
4,20800440,2008-12-27,HOU vs. UTA,0.006102,0,0,18.0,14.0,0.563,0.596774,0.485294,0.500000,0.559091,0.161290,0.151899,0.385,0.215686,0.269841,1.0,11.0,29.0,40,35.0,13.0,7.0,9.0,27.0,115,0.001953,2008,Regular Season,2008-09


In [8]:
X = df[['team_id', 'w_pct', 'a_team_id', 'is_home']]
y = df['wl']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=100)

Random Forest

In [ ]:
rforestclf = RandomForestClassifier()
rforestclf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
results = rforestclf.predict(X_test)

SVM

In [6]:
clf = make_pipeline(StandardScaler(), SVC(gamma=0.0001, class_weight='balanced'))
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=0.0001,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [7]:
results = clf.predict(X_test)

SVM Optimization

In [17]:
top = 0
max_tol = 0
gamma = 'auto'
max_kernel = 'linear'
kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']

for index in range(0, 4):
  clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel=kernels[index]))
  clf.fit(X_train, y_train)
  results = clf.predict(X_test)
  total = 0
  correct = 0
  for i in range(len(results)):
      total += 1
      if results[i] == y_test.ravel()[i]:
          correct += 1
  per = correct / total * 100
  print(per)
  if per > top:
    max_kernel = kernels[index]
    top = per

66.7196608373079
65.6673480202892
66.5606783253842
59.56544780074192


In [18]:
max_kernel, top

('linear', 66.7196608373079)

In [21]:
top = 0
max_tol = 0
gamma = 'auto'

for t in range(10, 100, 10):
  tol = t / 1000
  clf = make_pipeline(StandardScaler(), SVC(gamma='scale', tol=tol, kernel='linear'))
  clf.fit(X_train, y_train)
  results = clf.predict(X_test)
  total = 0
  correct = 0
  for i in range(len(results)):
      total += 1
      if results[i] == y_test.ravel()[i]:
          correct += 1
  per = correct / total * 100
  print(per)
  if per > top:
    max_tol = tol
    top = per

66.77265500794913
66.7196608373079
66.7196608373079
66.73480202891967
66.68180785827845
66.76508441214324
66.66666666666666
66.66666666666666
66.73480202891967


In [22]:
top, max_tol

(66.77265500794913, 0.01)

In [23]:
top = 0
max_tol = 0
gamma = 'auto'
max_kernel = 'linear'
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for t in range(10, 100, 10):
  for index in range(4):
    tol = t / 1000
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto', tol=tol, kernel=kernels[index]))
    clf.fit(X_train, y_train)
    results = clf.predict(X_test)
    total = 0
    correct = 0
    for i in range(len(results)):
        total += 1
        if results[i] == y_test.ravel()[i]:
            correct += 1
    per = correct / total * 100
    print(per)
    if per > top:
      max_tol = tol
      max_kernel = kernels[index]
      top = per

66.77265500794913
65.67491861609508
66.57581951699598
59.56544780074192
66.7196608373079
65.6673480202892
66.60610190021956
59.56544780074192
66.7196608373079
65.72791278673633
66.58339011280188
59.56544780074192
66.73480202891967
65.75062457415399
66.61367249602543
59.55787720493603
66.68180785827845
65.75062457415399
66.60610190021956
59.55787720493603
66.76508441214324
65.75062457415399
66.61367249602543
59.55787720493603
66.66666666666666
65.75062457415399
66.6288136876372
59.55787720493603
66.66666666666666
65.75062457415399
66.643954879249
59.55787720493603
66.73480202891967
65.75062457415399
66.61367249602543
59.56544780074192


In [24]:
top, max_kernel, max_tol

(66.77265500794913, 'linear', 0.01)

In [32]:
top = 0
max_tol = 0
gamma = 'auto'
max_kernel = 'linear'
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for t in range(10, 100, 10):
  for index in range(4):
    tol = t / 1000
    clf = make_pipeline(StandardScaler(), 
                        SVC(gamma='auto', 
                            tol=tol, 
                            kernel=kernels[index],
                            class_weight='balanced'))
    clf.fit(X_train, y_train)
    results = clf.predict(X_test)
    total = 0
    correct = 0
    for i in range(len(results)):
        total += 1
        if results[i] == y_test.ravel()[i]:
            correct += 1
    per = correct / total * 100
    print(per)
    if per > top:
      max_tol = tol
      max_kernel = kernels[index]
      top = per

66.7196608373079
65.6673480202892
66.57581951699598
59.550306609130146
66.74994322053146
65.65977742448331
66.54553713377243
59.550306609130146
66.76508441214324
65.6673480202892
66.5606783253842
59.550306609130146
66.71209024150201
65.6673480202892
66.58339011280188
59.54273601332425
66.73480202891967
65.68248921190097
66.59853130441365
59.54273601332425
66.73480202891967
65.68248921190097
66.62124309183133
59.54273601332425
66.7196608373079
65.68248921190097
66.6363842834431
59.54273601332425
66.72723143311379
65.68248921190097
66.643954879249
59.54273601332425
66.74237262472556
65.68248921190097
66.6288136876372
59.54273601332425


In [33]:
top, max_kernel, max_tol

(66.76508441214324, 'linear', 0.03)

Gamma Comparison

In [15]:
clf = make_pipeline(StandardScaler(), 
                        SVC(gamma='auto', 
                            tol=0.03, 
                            kernel='linear'))
clf.fit(X_train, y_train)
results = clf.predict(X_test)
total = 0
correct = 0
for i in range(len(results)):
    total += 1
    if results[i] == y_test.ravel()[i]:
        correct += 1
correct / total * 100

66.7196608373079

Testing

In [18]:
# Using the Optimized Model
clf = make_pipeline(StandardScaler(), 
                        SVC(gamma='auto', 
                            tol=0.03, 
                            kernel='linear'))
clf.fit(X_train, y_train)
results = clf.predict(X_test)

Baseline Testing (Error Rate by Team)

In [19]:
data = []
for result in results:
  threshold = 0.5
  if result < threshold:
    data.append(0)
  else:
    data.append(1)
results = data

In [34]:
teams = []
for index, row in X_test.iterrows():
  teams.append(index)

In [35]:
for i in range(len(teams)):
  index = teams[i]
  teams[i] = copied.loc[index, 'team_id']

In [36]:
team_wins = {}
team_losses = {}
for i in range(len(teams)):
  team = teams[i]
  if y_test.ravel()[i] == 1:
    if team not in team_wins:
      team_wins[team] = 1
    else:
      team_wins[team] += 1
  else:
    if team not in team_losses:
      team_losses[team] = 1
    else:
      team_losses[team] += 1

In [37]:
# Generate the Actual Win Percentage by Team
team_ratio = {}
for team in team_wins:
  team_ratio[team] = team_wins[team] / (team_wins[team] + team_losses[team])

In [26]:
team_ratio

{0.0: 0.42755344418052255,
 0.0002440810203552246: 0.5398230088495575,
 0.00048816209891811013: 0.5847826086956521,
 0.0007322431192733347: 0.448512585812357,
 0.0009763241396285594: 0.5247058823529411,
 0.001220405159983784: 0.5746102449888641,
 0.0014644862385466695: 0.58,
 0.001708567258901894: 0.5199063231850117,
 0.0019526482792571187: 0.597457627118644,
 0.0021967292996123433: 0.46405228758169936,
 0.002440810378175229: 0.5414847161572053,
 0.0026848913985304534: 0.5750528541226215,
 0.002928972418885678: 0.42630385487528344,
 0.0031730534392409027: 0.3613053613053613,
 0.003417134517803788: 0.4637002341920375,
 0.003661215538159013: 0.37857142857142856,
 0.0039052965585142374: 0.4617169373549884,
 0.004149377578869462: 0.5302013422818792,
 0.0043934586574323475: 0.41025641025641024,
 0.004637539677787572: 0.4845605700712589,
 0.004881620698142797: 0.4748201438848921,
 0.005125701718498021: 0.43855421686746987,
 0.005369782797060907: 0.689727463312369,
 0.0056138638174161315: 0.5

In [27]:
pred_wins = {}
pred_losses = {}
for i in range(len(teams)):
  team = teams[i]
  if results[i] == 1:
    if team not in pred_wins:
      pred_wins[team] = 1
    else:
      pred_wins[team] += 1
  else:
    if team not in pred_losses:
      pred_losses[team] = 1
    else:
      pred_losses[team] += 1

In [38]:
# Generate the Predicted Win Ratio By Team
pred_ratio = {}
for team in team_wins:
   pred_ratio[team] = pred_wins[team] / (pred_wins[team] + pred_losses[team])

In [30]:
pred_ratio

{0.0: 0.46318289786223277,
 0.0002440810203552246: 0.6261061946902655,
 0.00048816209891811013: 0.6391304347826087,
 0.0007322431192733347: 0.47368421052631576,
 0.0009763241396285594: 0.5105882352941177,
 0.001220405159983784: 0.6948775055679287,
 0.0014644862385466695: 0.58,
 0.001708567258901894: 0.5644028103044496,
 0.0019526482792571187: 0.6758474576271186,
 0.0021967292996123433: 0.5729847494553377,
 0.002440810378175229: 0.5829694323144105,
 0.0026848913985304534: 0.638477801268499,
 0.002928972418885678: 0.3854875283446712,
 0.0031730534392409027: 0.29836829836829837,
 0.003417134517803788: 0.32786885245901637,
 0.003661215538159013: 0.3547619047619048,
 0.0039052965585142374: 0.494199535962877,
 0.004149377578869462: 0.5771812080536913,
 0.0043934586574323475: 0.29603729603729606,
 0.004637539677787572: 0.5273159144893111,
 0.004881620698142797: 0.5227817745803357,
 0.005125701718498021: 0.327710843373494,
 0.005369782797060907: 0.8867924528301887,
 0.0056138638174161315: 0.58

In [39]:
# Generate the Percent Error by Team of Predicted - Actual
errors = {}
for team in team_ratio:
  errors[team] = abs(pred_ratio[team] - team_ratio[team]) / team_ratio[team] * 100

In [32]:
errors

{0.0: 8.333333333333336,
 0.0002440810203552246: 15.983606557377064,
 0.00048816209891811013: 9.293680297397772,
 0.0007322431192733347: 5.612244897959178,
 0.0009763241396285594: 2.690582959641242,
 0.001220405159983784: 20.93023255813953,
 0.0014644862385466695: 0.0,
 0.001708567258901894: 8.558558558558547,
 0.0019526482792571187: 13.120567375886527,
 0.0021967292996123433: 23.47417840375587,
 0.002440810378175229: 7.661290322580632,
 0.0026848913985304534: 11.029411764705891,
 0.002928972418885678: 9.57446808510638,
 0.0031730534392409027: 17.419354838709673,
 0.003417134517803788: 29.292929292929298,
 0.003661215538159013: 6.289308176100622,
 0.0039052965585142374: 7.035175879396978,
 0.004149377578869462: 8.860759493670894,
 0.0043934586574323475: 27.840909090909083,
 0.004637539677787572: 8.823529411764701,
 0.004881620698142797: 10.101010101010097,
 0.005125701718498021: 25.274725274725267,
 0.005369782797060907: 28.57142857142857,
 0.0056138638174161315: 10.699588477366252,
 0

In [33]:
total = 0
for error in errors:
  total += 1
total

32

In [40]:
# Find Overall Error for Generated Results
def accuracy(results, y_test):
  total = 0
  correct = 0
  for i in range(len(results)):
      total += 1
      if results[i] == y_test.ravel()[i]:
          correct += 1
  return correct / total * 100

KFold Testing

In [10]:
def accuracy_kfold(predictions, y_test):
  total = 0
  correct = 0
  for i in range(len(predictions.ravel())):
      pred = predictions.ravel()[i]
      total += 1
      if pred == y_test.to_numpy()[i]:
          correct += 1
  return correct / total * 100

In [11]:
splits = 25
kf = KFold(n_splits=splits)
X = df[['team_id', 'w_pct', 'a_team_id', 'is_home']]
X = X.to_numpy()
y = df['wl']
y = y.to_numpy()
kf.get_n_splits(X)
KFold(n_splits=splits, random_state=100, shuffle=True)
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  X_train = pd.DataFrame({'team_id': X_train[:, 0], 'w_pct': X_train[:, 1], 
                          'a_team_id': X_train[:, 2], 'is_home': X_train[:, 3]
                          })
  X_test = pd.DataFrame({'team_id': X_test[:, 0], 'w_pct': X_test[:, 1], 
                          'a_team_id': X_test[:, 2], 'is_home': X_test[:, 3]
                          })
  y_train = pd.DataFrame({'wl': y_train})
  y_test = pd.DataFrame({'wl': y_test})
  
  # Using the Optimized Model
  clf = make_pipeline(StandardScaler(), 
                          SVC(gamma='auto', 
                              tol=0.03, 
                              kernel='linear'))
  clf.fit(X_train, y_train)
  predictions = clf.predict(X_test)
  acc = accuracy_kfold(predictions, y_test)
  print(acc)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


68.27814569536423


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


67.28476821192054


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

67.79324055666004


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


66.73293571901921


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


66.99801192842942
